### インポート

In [30]:
 #-*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import gensim
from gensim import corpora, models
from sklearn.cluster import KMeans

### 事前付与ジャンルのリストを作成

In [31]:
pre_genre = ["csAI", "csCC", "csCG", "csCL", "csCV", "csDS", "csGT", "csMA", "csSD", "statAP", "statCO", "statME", "statML", "statTH"]

### CSV読み込み

In [32]:
df = pd.DataFrame()

for genre in pre_genre:
    df = pd.concat([df, pd.read_csv("../csv/" +  genre + ".csv", header=None)])

### データフレームにカラム名をつける

In [33]:
header_names =  ["プログラム実行日時", "論文更新日時", "論文リンク", "PDFリンク", "論文タイトル", "サマリ", "著者", "事前付与ジャンル"]

In [34]:
df.columns = header_names

### カテゴリおよびそれに対応する辞書

- 0: ニューラルネットワーク/ディープラーニング
- 1: 自然言語処理
- 2: マーケティング
- 3: 画像解析
- 4: 音声解析
- 5: 強化学習

In [35]:
cat_names = ["ニューラルネットワーク"
                     , "自然言語処理"
                     , "マーケティング"
                     , "画像解析"
                     , "音声解析"
                     , "強化学習"]

In [36]:
cat_kw = [
  ['neural'
    , 'rnn'
    , 'cnn'
    , 'deep'
    ],
  ['languag'
    , 'word'
    , 'document'
    , 'nlp'],
  ['market'
    , 'custom'
    , 'commerc'
    , 'advertis'
    ],
  ['imag'
    , 'vision'
    , 'photo'
    ],
  ['sound'
  , 'speech'],
  ['reinforc']
]

In [43]:
easy_cat = [] #簡易的なカテゴリ振り分けリスト

for n, row in df.iterrows():
    row_cat = []
    for i, cat in enumerate(cat_kw):
        flag = False
        for kw in cat:
            if kw in (row["論文タイトル"] + row["サマリ"]):
                flag = True
        if flag:
            row_cat.append(1)
        else:
            row_cat.append(0)
    easy_cat.append(row_cat)

df_cat = pd.DataFrame(easy_cat)

In [45]:
df.index=range(1400)

df_all = df.join(df_cat)
df_all.columns = header_names + cat_names
df_all
#df_all.to_csv("df_all.csv")

,プログラム実行日時,論文更新日時,論文リンク,PDFリンク,論文タイトル,サマリ,著者,事前付与ジャンル,ニューラルネットワーク,自然言語処理,マーケティング,画像解析,音声解析,強化学習
0,2017-03-06T15:29:38Z,2017-03-03T18:15:32Z,http://arxiv.org/abs/1703.01274v1,http://arxiv.org/pdf/1703.01274v1,actor critic reinforc learn simultan human con...,paper contribut first studi differ human user ...,Kory W. Mathewson|Patrick M. Pilarski,cs.AI|cs.HC|cs.RO,0,0,0,0,0,1
1,2017-03-06T15:29:38Z,2017-03-03T15:27:38Z,http://arxiv.org/abs/1703.01203v1,http://arxiv.org/pdf/1703.01203v1,stochast separ theorem,set linear separ separ rest linear function st...,A. N. Gorban|I. Y. Tyukin,cs.LG|cs.AI|68T10|I.2.6,0,0,0,0,0,0
2,2017-03-06T15:29:38Z,2017-03-03T14:05:11Z,http://arxiv.org/abs/1703.01161v1,http://arxiv.org/pdf/1703.01161v1,feudal network hierarch reinforc learn,introduc feudal network fun novel architectur ...,Alexander Sasha Vezhnevets|Simon Osindero|Tom ...,cs.AI,1,0,0,0,0,1
3,2017-03-06T15:29:38Z,2017-03-03T12:23:13Z,http://arxiv.org/abs/1703.01127v1,http://arxiv.org/pdf/1703.01127v1,behavior convolut net featur extract,convolut neural network cnn represent learn te...,Dario Garcia-Gasulla|Ferran Parés|Armand Vilal...,cs.NE|cs.AI|cs.LG|stat.ML,1,1,0,1,0,0
4,2017-03-06T15:29:38Z,2017-03-03T09:03:46Z,http://arxiv.org/abs/1703.01083v1,http://arxiv.org/pdf/1703.01083v1,sequenti plan recognit,plan recognit algorithm infer agent plan obser...,Reuth Mirsky|Roni Stern|Ya'akov|Gal|Meir Kalech,cs.AI,0,0,0,0,0,0
5,2017-03-06T15:29:38Z,2017-03-03T05:41:30Z,http://arxiv.org/abs/1703.01041v1,http://arxiv.org/pdf/1703.01041v1,larg scale evolut imag classifi,neural network proven effect solv difficult pr...,Esteban Real|Sherry Moore|Andrew Selle|Saurabh...,cs.NE|cs.AI|cs.CV|cs.DC|I.2.6; I.5.1; I.5.2,1,0,0,1,0,0
6,2017-03-06T15:29:38Z,2017-03-03T05:27:50Z,http://arxiv.org/abs/1703.01040v1,http://arxiv.org/pdf/1703.01040v1,learn robot activ first person human video use...,design new approach allow robot learn new acti...,Jangwon Lee|Michael S. Ryoo,cs.RO|cs.AI|cs.CV|cs.LG,1,0,0,1,0,0
7,2017-03-06T15:29:38Z,2017-03-03T03:24:03Z,http://arxiv.org/abs/1703.01026v1,http://arxiv.org/pdf/1703.01026v1,unsupervis basi function adapt reinforc learn,use reinforc learn rl algorithm evalu polici c...,Edward W. Barker|Charl J. Ras,cs.AI|cs.LG|stat.ML,0,0,0,0,0,1
8,2017-03-06T15:29:38Z,2017-03-03T01:29:11Z,http://arxiv.org/abs/1703.01008v1,http://arxiv.org/pdf/1703.01008v1,end end task complet neural dialogu system,paper present end end learn framework task com...,Xuijun Li|Yun-Nung Chen|Lihong Li|Jianfeng Gao,cs.CL|cs.AI,1,0,0,0,0,1
9,2017-03-06T15:29:38Z,2017-03-02T21:31:29Z,http://arxiv.org/abs/1703.00956v1,http://arxiv.org/pdf/1703.00956v1,laplacian framework option discoveri reinforc ...,represent learn option discoveri two biggest c...,Marlos C. Machado|Marc G. Bellemare|Michael Bo...,cs.LG|cs.AI,0,0,0,0,0,1


In [48]:
df_cat_list = []

for cat in cat_names:
    tmp = df_all[df_all[cat] == 1]
    df_cat_list.append(tmp)

df_cat_list[0]

,プログラム実行日時,論文更新日時,論文リンク,PDFリンク,論文タイトル,サマリ,著者,事前付与ジャンル,ニューラルネットワーク,自然言語処理,マーケティング,画像解析,音声解析,強化学習
2,2017-03-06T15:29:38Z,2017-03-03T14:05:11Z,http://arxiv.org/abs/1703.01161v1,http://arxiv.org/pdf/1703.01161v1,feudal network hierarch reinforc learn,introduc feudal network fun novel architectur ...,Alexander Sasha Vezhnevets|Simon Osindero|Tom ...,cs.AI,1,0,0,0,0,1
3,2017-03-06T15:29:38Z,2017-03-03T12:23:13Z,http://arxiv.org/abs/1703.01127v1,http://arxiv.org/pdf/1703.01127v1,behavior convolut net featur extract,convolut neural network cnn represent learn te...,Dario Garcia-Gasulla|Ferran Parés|Armand Vilal...,cs.NE|cs.AI|cs.LG|stat.ML,1,1,0,1,0,0
5,2017-03-06T15:29:38Z,2017-03-03T05:41:30Z,http://arxiv.org/abs/1703.01041v1,http://arxiv.org/pdf/1703.01041v1,larg scale evolut imag classifi,neural network proven effect solv difficult pr...,Esteban Real|Sherry Moore|Andrew Selle|Saurabh...,cs.NE|cs.AI|cs.CV|cs.DC|I.2.6; I.5.1; I.5.2,1,0,0,1,0,0
6,2017-03-06T15:29:38Z,2017-03-03T05:27:50Z,http://arxiv.org/abs/1703.01040v1,http://arxiv.org/pdf/1703.01040v1,learn robot activ first person human video use...,design new approach allow robot learn new acti...,Jangwon Lee|Michael S. Ryoo,cs.RO|cs.AI|cs.CV|cs.LG,1,0,0,1,0,0
8,2017-03-06T15:29:38Z,2017-03-03T01:29:11Z,http://arxiv.org/abs/1703.01008v1,http://arxiv.org/pdf/1703.01008v1,end end task complet neural dialogu system,paper present end end learn framework task com...,Xuijun Li|Yun-Nung Chen|Lihong Li|Jianfeng Gao,cs.CL|cs.AI,1,0,0,0,0,1
10,2017-03-06T15:29:41Z,2017-03-02T21:23:47Z,http://arxiv.org/abs/1703.00955v1,http://arxiv.org/pdf/1703.00955v1,control text generat,generic generat manipul text challeng limit su...,Zhiting Hu|Zichao Yang|Xiaodan Liang|Ruslan Sa...,cs.LG|cs.AI|cs.CL|stat.ML,1,1,0,0,0,0
17,2017-03-06T15:29:41Z,2017-03-01T23:40:42Z,http://arxiv.org/abs/1703.00548v1,http://arxiv.org/pdf/1703.00548v1,evolv deep neural network,success deep learn depend find architectur fit...,Risto Miikkulainen|Jason Liang|Elliot Meyerson...,cs.NE|cs.AI,1,1,0,1,0,0
21,2017-03-06T15:29:46Z,2017-03-01T18:58:48Z,http://arxiv.org/abs/1703.00443v1,http://arxiv.org/pdf/1703.00443v1,optnet differenti optim layer neural network,paper present optnet network architectur integ...,Brandon Amos|J. Zico Kolter,cs.LG|cs.AI|math.OC|stat.ML,1,0,0,0,0,0
22,2017-03-06T15:29:46Z,2017-03-01T18:52:23Z,http://arxiv.org/abs/1703.00441v1,http://arxiv.org/pdf/1703.00441v1,learn optim neural net,learn optim recent propos framework learn opti...,Ke Li|Jitendra Malik,cs.LG|cs.AI|math.OC|stat.ML,1,0,0,0,0,1
24,2017-03-06T15:29:46Z,2017-03-01T18:20:19Z,http://arxiv.org/abs/1703.00426v1,http://arxiv.org/pdf/1703.00426v1,holstep machin learn dataset higher order logi...,larg comput understand proof consist million i...,Cezary Kaliszyk|François Chollet|Christian Sze...,cs.AI,1,0,0,0,0,0


In [53]:
df_cat = df_cat_list[0]

documents = df_cat["サマリ"]
texts = [[word for word in document.lower().split()]for document in documents] #単語ごとに抽出
dictionary = corpora.Dictionary(texts)

239

In [18]:
for df_cat in df_cat_list:
    documents = df_cat["サマリ"]
    # create dict
    texts = [[word for word in document.lower().split()]for document in documents] #単語ごとに抽出
    dict = corpora.Dictionary(texts)

### 辞書作成

In [20]:
dictionary = corpora.Dictionary(texts)

### 辞書のID数を確認

In [21]:
id_num = len(dictionary.token2id)

### Bag of words のコーパスを作成

In [22]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [23]:
tfidf = models.TfidfModel(corpus)

In [24]:
print (tfidf)

TfidfModel(num_docs=1400, num_nnz=12859)


In [25]:
corpus_tfidf = tfidf[corpus]

### ベクトル化

In [26]:
num_topics = 6
lsi = gensim.models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=num_topics) # initialize an LSI transformation
pprint(lsi.print_topics(10))
print(lsi.num_topics)
corpus_lsi = lsi[corpus_tfidf]

In [ ]:
vecs = gensim.matutils.corpus2dense(corpus_lsi, num_terms=1400) #スパースなリスト形式を密にしてsklearnに適用可能な形に
pprint(vecs)

### k-means クラスタリング

In [27]:
clusters = KMeans(n_clusters=6, random_state=0).fit_predict(vecs)

In [28]:
cls_list = [0, 0, 0, 0, 0, 0]

for doc, cls in zip(documents, clusters):
    #print (cls, doc)
    cls_list[cls] += 1

print (cls_list)

[11, 8, 7, 4, 1, 1369]
